In [1]:
data_var = '2024-01-25'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10840,2024-01-25,Brasil Nbb,20:00,Flamengo,União Corinthians,1.04,9.45,158.5,1.85,1.85,-18.5,2.08,1.66,ncUg9ANB,0.961538,0.105820,0.540541,0.540541,0.067359,0.6,0.6,NaN,NaN,107.620,19.027108,0.176799,301.454,139.375316,0.462344,117.166,289.998,114.45,306.46,0.0,0.0,0.0,0.0,1.133798,0.000000,0.158815,1.37,0.274,0.145985,0.000000,0.0,0.000000,-2.47,-0.494,-17.105263,0.318433,0.3,-0.018433
10841,2024-01-25,Eua Nba,00:00,Golden State Warriors,Atlanta Hawks,1.46,3.04,234.5,1.88,2.04,-6.5,2.06,1.87,p6wpDlC0,0.684932,0.328947,0.531915,0.490196,0.013879,0.8,0.4,NaN,NaN,295.536,212.682328,0.719649,255.418,119.277121,0.466988,212.328,291.860,214.76,403.39,0.0,0.0,0.0,0.0,0.496546,0.057723,0.068372,-2.11,-0.422,-1.090047,0.627272,0.6,-0.027272,-0.25,-0.050,-40.800000,0.444809,0.4,-0.044809
10842,2024-01-25,Eua Nba,21:00,Indiana Pacers,Philadelphia 76ers,2.64,1.49,236.5,1.84,1.97,3.5,2.05,1.77,OdxlC8R6,0.378788,0.671141,0.543478,0.507614,0.049929,0.4,0.4,NaN,NaN,307.000,117.573891,0.382977,174.728,47.729108,0.273162,244.538,214.060,258.33,118.34,0.0,0.0,0.0,0.0,0.393788,0.048254,0.103660,0.31,0.062,26.451613,0.627721,0.6,-0.027721,0.04,0.008,61.250000,0.598343,0.6,0.001657
10843,2024-01-25,Eua Nba,21:00,Washington Wizards,Utah Jazz,3.44,1.38,243.5,1.97,1.97,5.5,2.10,1.67,tYWhBStD,0.290698,0.724638,0.507614,0.507614,0.015335,0.4,0.4,NaN,NaN,535.596,217.696121,0.406456,277.324,92.525934,0.333638,474.208,548.128,661.26,415.40,1.0,0.0,0.0,0.0,0.604415,0.000000,0.161303,-5.00,-1.000,-2.440000,0.300790,0.2,-0.100790,1.37,0.274,1.386861,0.322638,0.4,0.077362
10844,2024-01-25,Eua Nba,21:30,Brooklyn Nets,Minnesota Timberwolves,2.57,1.60,218.5,1.80,1.95,2.5,2.10,1.67,KOVdAndJ,0.389105,0.625000,0.555556,0.512821,0.014105,0.6,0.2,NaN,NaN,297.308,149.473825,0.502757,142.476,13.487358,0.094664,249.906,168.082,263.68,138.06,0.0,0.0,0.0,0.0,0.328966,0.056569,0.161303,-2.12,-0.424,-3.702830,0.551001,0.4,-0.151001,0.50,0.100,6.000000,0.593184,0.6,0.006816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10946,2024-01-25,Israel Wbl Feminina,12:00,Holon F,Hapoel Petah Tikva F,1.65,2.15,130.5,1.85,1.85,-2.5,1.86,1.84,K4ymmm5s,0.606061,0.465116,0.540541,0.540541,0.071177,0.0,0.0,NaN,NaN,NaN,NaN,NaN,278.836,238.716304,0.856117,0.000,0.000,64.38,683.10,0.0,1.0,1.0,0.0,0.186081,0.000000,0.007644,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10947,2024-01-25,Israel Wbl Feminina,13:00,Hapoel Jerusalem F,Maccabi Ashdod F,3.02,1.35,136.5,1.85,1.85,5.5,1.95,1.72,lMpHh9sK,0.331126,0.740741,0.540541,0.540541,0.071867,0.0,0.0,NaN,NaN,152.412,125.070473,0.820608,212.440,156.150522,0.735034,0.000,0.000,108.12,97.11,0.0,0.0,0.0,0.0,0.540443,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10948,2024-01-25,Israel Wbl Feminina,14:00,Asa Jerusalem F,Maccabi Urban F,8.10,1.06,132.5,1.85,1.85,12.5,1.95,1.72,IekLiTdQ,0.123457,0.943396,0.540541,0.540541,0.066853,0.0,0.0,NaN,NaN,NaN,NaN,NaN,101.314,42.411020,0.418610,0.000,0.000,451.44,172.97,1.0,0.0,0.0,0.0,1.086906,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10949,2024-01-25,Israel Wbl Feminina,14:00,Hapoel Rishon LeZion F,Ramat Hasharon F,1.35,3.02,132.5,1.85,1.85,-7.5,1.95,1.72,8

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Atlanta Hawks,234.5,1.88,1.0000,Over
1,21:30,Eua Nba,Miami Heat,Boston Celtics,223.5,1.82,0.9960,Over
2,16:05,Europa Euroliga,Maccabi Tel Aviv,Panathinaikos,166.5,1.87,0.7194,Over
3,16:45,Europa Euroliga,Real Madrid,Olympiakos,160.5,1.80,1.0000,Over
4,21:30,Eua Ncaa,Stephen F. Austin,Grand Canyon,151.5,1.91,0.8422,Over
